In [95]:
#############################################################################
### Търсене и извличане на информация. Приложение на дълбоко машинно обучение
### Стоян Михов
### Зимен семестър 2021/2022
#############################################################################

### Домашно задание 1
###
### Преди да се стартира програмата е необходимо да се активира съответното обкръжение с командата:
### conda activate tii
###
### Ако все още нямате създадено обкръжение прочетете файла README.txt за инструкции


########################################
import numpy as np
import random
import re

alphabet = ['а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ь', 'ю', 'я']

def extractDictionary(corpus):
    dictionary = set()
    for doc in corpus:
        for w in doc:
            if w not in dictionary: dictionary.add(w)
    return dictionary

def editDistance(s1, s2):
    #### функцията намира разстоянието на Левенщайн-Дамерау между два низа
    #### вход: низовете s1 и s2
    #### изход: минималният брой на елементарните операции ( вмъкване, изтриване, субституция и транспоциция на символи) необходими, за да се получи от единия низ другия
    
    #############################################################################
    #### Начало на Вашия код. На мястото на pass се очакват 10-25 реда

    lenstr1 = len(s1)
    lenstr2 = len(s2)
    adjMatrix = [[0 for j in range(0,lenstr2+1)]for i in range(0,lenstr1+1)]
    for i in range(0,lenstr1+1):
        adjMatrix[i][0] = i
    for j in range(0,lenstr2+1):
        adjMatrix[0][j] = j

    for i in range(1,lenstr1+1):
        for j in range(1,lenstr2+1):
            if s1[i-1] == s2[j-1]:
                cost = 0
            else:
                cost = 1
            adjMatrix[i][j] = min(
                           adjMatrix[i-1][j] + 1,
                           adjMatrix[i][j-1] + 1,
                           adjMatrix[i-1][j-1] + cost,
                          ) 
            if i > 1 and j > 1 and s1[i-1]==s2[j-2] and s1[i-2] == s2[j-1]:
                adjMatrix[i][j] = min (adjMatrix[i][j], adjMatrix[i-2][j-2] + cost)

    return adjMatrix[lenstr1][lenstr2]
    #### Край на Вашия код
    #############################################################################

def getLDMAtrix(s1, s2):
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    adjMatrix = [[0 for j in range(0,lenstr2+1)]for i in range(0,lenstr1+1)]
    for i in range(0,lenstr1+1):
        adjMatrix[i][0] = i
    for j in range(0,lenstr2+1):
        adjMatrix[0][j] = j

    for i in range(1,lenstr1+1):
        for j in range(1,lenstr2+1):
            if s1[i-1] == s2[j-1]:
                cost = 0
            else:
                cost = 1
            adjMatrix[i][j] = min(
                           adjMatrix[i-1][j] + 1,
                           adjMatrix[i][j-1] + 1,
                           adjMatrix[i-1][j-1] + cost,
                          ) 
            if i > 1 and j > 1 and s1[i-1]==s2[j-2] and s1[i-2] == s2[j-1]:
                adjMatrix[i][j] = min (adjMatrix[i][j], adjMatrix[i-2][j-2] + cost)

    return adjMatrix

def getOperationsList(adjMatrix, s1, s2):
    operations = []
    i = len(s1)
    j = len(s2)
    while i!=0 and j!=0:
        if i > 1 and j > 1 and s1[i-1] == s2[j-2] and s1[i-2] == s2[j-1]:
                if adjMatrix[i-2][j-2] < adjMatrix[i][j]:
                    operations.append("trans")
                    i -= 2
                    j -= 2
                    continue
        if s1[i-1] == s2[j-1]:
            cost = 0
        else:
            cost = 1
        backtrace = [adjMatrix[i-1][j-1]+cost, adjMatrix[i][j-1]+1, adjMatrix[i-1][j]+1]
        index = backtrace.index(min(backtrace))
        if backtrace[index] == backtrace[2]:
            index = 2
        if index == 0:
            if adjMatrix[i][j] > adjMatrix[i-1][j-1]:
                operations.append("sub")
            else:
                operations.append("id")
            i -= 1
            j -= 1
        elif index == 1:
            operations.append("ins")
            j -= 1
        elif index == 2:
            operations.append("del")
            i -= 1
    if i > 0:
        while i!=0:
            operations.append("del")
            i-=1
    if j > 0:
        while j!=0:
            operations.append("ins")
            j-=1
    operations.reverse()
    return operations

def editOperations(s1, s2):
    #### функцията намира елементарни редакции, неободими за получаването на един низ от друг
    #### вход: низовете s1 и s2
    #### изход: списък с елементарните редакции ( идентитет, вмъкване, изтриване, субституция и транспоциция на символи) необходими, за да се получи втория низ от първия
    
    #### Например: editOperations('котка', 'октава') би следвало да връща списъка:
    ####    [('ко', 'ок'), ('т','т'), ('', 'а'), ('к', 'в'), ('а','а')]
    ####        |ко   |т |   |к  |а |
    ####        |ок   |т |а  |в  |а |
    ####        |Trans|Id|Ins|Sub|Id|
    ####
    #### Можете да преизползвате и модифицирате кода на функцията editDistance
    #############################################################################
    #### Начало на Вашия код.

    adjMatrix = getLDMAtrix(s1, s2)
    listOfOperations = getOperationsList(adjMatrix, s1, s2)
    i = 0
    j = 0
    operations = []
    for op in listOfOperations:
        if op == "id" or op == "sub":
            operations.append((s1[i], s2[j]))
            i+=1
            j+=1
        elif op == "ins":
            operations.append(('', s2[j]))
            j+=1
        elif op == "del":
            operations.append((s1[i], ''))
            i+=1
        elif op == 'trans':
            operations.append((s1[i]+s1[i+1], s2[j]+s2[j+1]))
            i+=2
            j+=2    
    return operations
    #### Край на Вашия код
    #############################################################################
#print(editOperations('заявката', 'заявката'))
#print(editOperations('заявката', 'заявьата'))
#print(editOperations('заявката', 'завякатва'))
#print(editOperations('заявката', 'заявкатаа'))
#print(editOperations('заявката', 'вя'))
#print(editOperations('заявката', 'язвката'))

def computeOperationProbs(corrected_corpus,uncorrected_corpus,smoothing = 0.2):
    #### Функцията computeOperationProbs изчислява теглата на дадени елементарни операции (редакции)
    #### Теглото зависи от конкретните символи. Използвайки корпусите, извлечете статистика. Използвайте принципа за максимално правдоподобие. Използвайте изглаждане. 
    #### Вход: Корпус без грешки, Корпус с грешки, параметър за изглаждане. С цел простота може да се счете, че j-тата дума в i-тото изречение на корпуса с грешки е на разстояние не повече от 2 (по Левенщайн-Дамерау) от  j-тата дума в i-тото изречение на корпуса без грешки.
    #### Следва да се използват функциите generateCandidates, editOperations, 
    #### Помислете как ще изберете кандидат за поправка измежду всички възможни.
    #### Важно! При изтриване и вмъкване се предполага, че празния низ е представен с ''
    #### Изход: Речник, който по зададена наредена двойка от низове връща теглото за операцията.
    
    #### Първоначално ще трябва да преброите и запишете в речника operations броя на редакциите от всеки вид нужни, за да се поправи корпуса с грешки. След това изчислете съответните вероятности.
    
    operations = {} # Брой срещания за всяка елементарна операция + изглаждане
    operationsProb = {} # Емпирична вероятност за всяка елементарна операция
    for c in alphabet:
        operations[(c,'')] = smoothing    # deletions
        operations[('',c)] = smoothing    # insertions
        for s in alphabet:
            operations[(c,s)] = smoothing    # substitution and identity
            if c == s:    
                continue
            operations[(c+s,s+c)] = smoothing    # transposition
    #############################################################################
    #### Начало на Вашия код.
    
    for r1, r2 in zip(corrected_corpus, uncorrected_corpus):
       for s1, s2 in zip(r1, r2):
            ops = editOperations(s1, s2)
            for op in ops:
                if op in operations:
                    operations[op] += 1

    opSetSize = len(operations)
    countOp = sum(int(o) if o > 0.2 else 0 for o in operations.values())
    for op, val in operations.items():
        operationsProb[op] = val/(smoothing*opSetSize + countOp)

    #### Край на Вашия код.
    #############################################################################
    return operationsProb

def operationWeight(a,b,operationProbs):
    #### Функцията operationWeight връща теглото на дадена елементарна операция
    #### Вход: Двата низа a,b, определящи операцията.
    ####       Речник с вероятностите на елементарните операции.
    #### Важно! При изтриване и вмъкване се предполага, че празния низ е представен с ''
    #### изход: Теглото за операцията
    
    if (a,b) in operationProbs.keys():
        return -np.log(operationProbs[(a,b)])
    else:
        print("Wrong parameters ({},{}) of operationWeight call encountered!".format(a,b))

def editWeight(s1, s2, operationProbs):
    #### функцията editWeight намира теглото между два низа
    #### За намиране на елеметарните тегла следва да се извиква функцията operationWeight
    #### вход: низовете s1 и s2 и речник с вероятностите на елементарните операции.
    #### изход: минималното тегло за подравняване, за да се получи втория низ от първия низ
    
    #############################################################################
    #### Начало на Вашия код. На мястото на pass се очакват 10-25 реда
    weight = 0
    for op in editOperations(s1, s2):
        weight += operationWeight(op[0], op[1], operationProbs)
    return weight

    #### Край на Вашия код. 
    #############################################################################

def generateEdits(q):
    ### помощната функция, generateEdits по зададена заявка генерира всички възможни редакции на разстояние едно от тази заявка.
    ### Вход: заявка като низ q
    ### Изход: Списък от низове на разстояние 1 по Левенщайн-Дамерау от заявката
    ###
    ### В тази функция вероятно ще трябва да използвате азбука, която е дефинирана с alphabet
    ###
    #############################################################################
    #### Начало на Вашия код. На мястото на pass се очакват 10-15 реда

    l = list()
    for i in range(len(q)+1):
        for j in range(len(alphabet)):
            l.append(q[:i]+alphabet[j]+q[i:])
    
    for i in range(len(q)):
        l.append(q[:i]+q[i+1:])
        for j in range(len(alphabet)):
            if(q[i]!=alphabet[j]):
                l.append(q[:i]+alphabet[j]+q[i+1:])

    for i in range(1, len(q)):
        l.append(q[:i-1]+q[i]+q[i-1]+q[i+1:])

    return l
    '''
    words = np.empty(0, dtype='U{}'.format(len(q)+1))
    for i in range(len(q)+1):
        words = np.append(words, q[:i]+q[i+1:])
        words = np.append(words, q[:i]+q[i:i+2][::-1]+q[i+2:])
        words = np.append(words, [[q[:i]+c+q[i:],q[:i]+c+q[i+1:]] for c in alphabet])
    
    return np.unique(np.delete(words, np.where(words == q))).tolist()
    '''
    #### Край на Вашия код
    #############################################################################

def generateCandidates(query,dictionary,operationProbs):
    ### Започва от заявката query и генерира всички низове НА РАЗСТОЯНИЕ <= 2, за да се получат кандидатите за корекция. Връщат се единствено кандидати, които са в речника dictionary.
        
    ### Вход:
    ###     Входен низ query
    ###     Речник с допустими (правилни) думи: dictionary
    ###     речник с вероятностите на елементарните операции.

    ### Изход:
    ###     Списък от двойки (candidate, candidate_edit_log_probability), където candidate е низ на кандидат, а candidate_edit_log_probability е логаритъм от вероятността за редакция -- минус теглото.
    
    #############################################################################
    #### Начало на Вашия код. На мястото на pass се очакват 10-15 реда

    distance1 = generateEdits(query)
    setOfDistances = set(distance1)

    for dist in distance1:
        setOfDistances = set.union(setOfDistances, generateEdits(dist))
    
    instersectedWords = set.intersection(dictionary, setOfDistances)
    l = []
    for w in instersectedWords:
        l.append((w, editWeight(w, query, operationProbs)))

    return l

    #### Край на Вашия код
    #############################################################################

def correctSpelling(r, dictionary, operationProbs):
    ### Функцията поправя корпус съдържащ евентуално сгрешени думи
    ### Генераторът на кандидати връща и вероятността за редактиране.
    ###
    ### Вход:
    ###    заявка: r - корпус от думи
    ###    речник с правилни думи: dictionary,
    ###    речник с вероятностите на елементарните операции: operationProbs
    ###    Изход: поправен корпус

    #############################################################################
    #### Начало на Вашия код. На мястото на pass се очакват 5-15 реда

    l = []
    for row in r:
        for w in row:
            matched_list = [characters in alphabet for characters in w]
            if(all(matched_list)):
                s = generateCandidates(w, dictionary, operationProbs)
                if len(s) != 0:
                    minTuple = min(s, key=lambda tup:tup[1])
                l.append(minTuple[0])
            else:
                l.append(w)
    return [l]
    #### Край на Вашия код
    #############################################################################


In [33]:
#############################################################################
### Търсене и извличане на информация. Приложение на дълбоко машинно обучение
### Стоян Михов
### Зимен семестър 2021/2022
#############################################################################

### Домашно задание 1
###
### Преди да се стартира програмата е необходимо да се активира съответното обкръжение с командата:
### conda activate tii
###
### Ако все още нямате създадено обкръжение прочетете файла README.txt за инструкции


########################################
import numpy as np
import random

alphabet = ['а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ь', 'ю', 'я']

def extractDictionary(corpus):
    dictionary = set()
    for doc in corpus:
        for w in doc:
            if w not in dictionary: dictionary.add(w)
    return dictionary

def editDistance(s1, s2):
    #### функцията намира разстоянието на Левенщайн-Дамерау между два низа
    #### вход: низовете s1 и s2
    #### изход: минималният брой на елементарните операции ( вмъкване, изтриване, субституция и транспоциция на символи) необходими, за да се получи от единия низ другия

    #############################################################################
    #### Начало на Вашия код. На мястото на pass се очакват 10-25 реда

    distMatrix = [[y+x for x in range(len(s2)+1)] for y in range(len(s1)+1)]
    
    for i in range(1,len(s1)+1):
        for j in range(1,len(s2)+1):
            cost = 0 if s1[i-1]==s2[j-1] else 1
            distMatrix[i][j] = min(distMatrix[i-1][j-1] + cost,
                                distMatrix[i][j-1] + 1,
                                distMatrix[i-1][j] + 1)
            if i>1 and j>1 and s1[i-1]==s2[j-2] and s1[i-2]==s2[j-1] and s1[i-1]!=s2[j-1]:
                distMatrix[i][j] = min(distMatrix[i][j],
                                distMatrix[i-2][j-2] + 1)
    return distMatrix[-1][-1]

    #### Край на Вашия код
    #############################################################################
    
def editOperations(s1, s2):
    #### функцията намира елементарни редакции, неободими за получаването на един низ от друг
    #### вход: низовете s1 и s2
    #### изход: списък с елементарните редакции ( идентитет, вмъкване, изтриване, субституция и транспоциция на символи) необходими, за да се получи втория низ от първия
    
    #### Например: editOperations('котка', 'октава') би следвало да връща списъка:
    ####    [('ко', 'ок'), ('т','т'), ('', 'а'), ('к', 'в'), ('а','а')]
    ####        |ко   |т |   |к  |а |
    ####        |ок   |т |а  |в  |а |
    ####        |Trans|Id|Ins|Sub|Id|
    ####
    #### Можете да преизползвате и модифицирате кода на функцията editDistance
    #############################################################################
    #### Начало на Вашия код.

    len1 = len(s1) + 1; len2 = len(s2) + 1
    distMatrix = [[0 for x in range(len2)] for y in range(len1)]
    for i in range(len2):
        distMatrix[0][i] = (i,0,max(i-1,0),'',s2[i-1] if i > 0 else '')
    for i in range(len1):
        distMatrix[i][0] = (i,max(i-1,0),0,s1[i-1] if i > 0 else '','')

    for i in range(1,len1):
        for j in range(1,len2):
            cost = 0 if s1[i-1]==s2[j-1] else 1
            distMatrix[i][j] = min((distMatrix[i-1][j-1][0] + cost, i-1, j-1, s1[i-1], s2[j-1]),
                                (distMatrix[i][j-1][0] + 1, i, j-1, '', s2[j-1]),
                                (distMatrix[i-1][j][0] + 1, i-1, j, s1[i-1], ''),
                                key = lambda x: x[0])
            if i>1 and j>1 and s1[i-1]==s2[j-2] and s1[i-2]==s2[j-1] and s1[i-1]!=s2[j-1]:
                distMatrix[i][j] = min(distMatrix[i][j],
                                (distMatrix[i-2][j-2][0] + 1, i-2, j-2, s1[i-2:i], s2[j-2:j]),
                                key = lambda x: x[0])
                
    result = []
    cell = (len(s1), len(s2))
    while any(cell):
        result += [distMatrix[cell[0]][cell[1]][3:]]
        cell = distMatrix[cell[0]][cell[1]][1:3]
    return result[::-1]

    #### Край на Вашия код
    #############################################################################

def computeOperationProbs(corrected_corpus,uncorrected_corpus,smoothing = 0.2):
    #### Функцията computeOperationProbs изчислява теглата на дадени елементарни операции (редакции)
    #### Теглото зависи от конкретните символи. Използвайки корпусите, извлечете статистика. Използвайте принципа за максимално правдоподобие. Използвайте изглаждане. 
    #### Вход: Корпус без грешки, Корпус с грешки, параметър за изглаждане. С цел простота може да се счете, че j-тата дума в i-тото изречение на корпуса с грешки е на разстояние не повече от 2 (по Левенщайн-Дамерау) от  j-тата дума в i-тото изречение на корпуса без грешки.
    #### Следва да се използват функциите generateCandidates, editOperations, 
    #### Помислете как ще изберете кандидат за поправка измежду всички възможни.
    #### Важно! При изтриване и вмъкване се предполага, че празния низ е представен с ''
    #### Изход: Речник, който по зададена наредена двойка от низове връща теглото за операцията.
    
    #### Първоначално ще трябва да преброите и запишете в речника operations броя на редакциите от всеки вид нужни, за да се поправи корпуса с грешки. След това изчислете съответните вероятности.
    
    operations = {} # Брой срещания за всяка елементарна операция + изглаждане
    operationsProb = {} # Емпирична вероятност за всяка елементарна операция
    for c in alphabet:
        operations[(c,'')] = smoothing    # deletions
        operations[('',c)] = smoothing    # insertions
        for s in alphabet:
            operations[(c,s)] = smoothing    # substitution and identity
            if c == s:    
                continue
            operations[(c+s,s+c)] = smoothing    # transposition

    #############################################################################
    #### Начало на Вашия код.

    for sent1,sent2 in zip(corrected_corpus, uncorrected_corpus):
        for word1,word2 in zip(sent1,sent2):
            for op in editOperations(word1, word2):
                if op in operations:
                    operations[op] += 1

    operationsCorpus = sum(int(o) for o in operations.values())
    for op,val in operations.items():
        operationsProb[op] = val/(smoothing*len(operations)+operationsCorpus)
    
    #### Край на Вашия код.
    #############################################################################
    return operationsProb

def operationWeight(a,b,operationProbs):
    #### Функцията operationWeight връща теглото на дадена елементарна операция
    #### Вход: Двата низа a,b, определящи операцията.
    ####       Речник с вероятностите на елементарните операции.
    #### Важно! При изтриване и вмъкване се предполага, че празния низ е представен с ''
    #### изход: Теглото за операцията
    
    if (a,b) in operationProbs.keys():
        return -np.log(operationProbs[(a,b)])
    else:
        print("Wrong parameters ({},{}) of operationWeight call encountered!".format(a,b))

def editWeight(s1, s2, operationProbs):
    #### функцията editWeight намира теглото между два низа
    #### За намиране на елеметарните тегла следва да се извиква функцията operationWeight
    #### вход: низовете s1 и s2 и речник с вероятностите на елементарните операции.
    #### изход: минималното тегло за подравняване, за да се получи втория низ от първия низ
    
    #############################################################################
    #### Начало на Вашия код. На мястото на pass се очакват 10-25 реда

    len1 = len(s1) + 1; len2 = len(s2) + 1
    distMatrix = [[0 for x in range(len2)] for y in range(len1)]
    for i in range(1, len2):
        distMatrix[0][i] += operationWeight('', s2[i-1], operationProbs) + distMatrix[0][i-1]
    for i in range(1, len1):
        distMatrix[i][0] += operationWeight(s1[i-1], '', operationProbs) + distMatrix[i-1][0]

    for i in range(1,len1):
        for j in range(1,len2):
            distMatrix[i][j] = min(distMatrix[i-1][j-1] + operationWeight(s1[i-1], s2[j-1], operationProbs),
                                distMatrix[i][j-1] + operationWeight('', s2[j-1], operationProbs),
                                distMatrix[i-1][j] + operationWeight(s1[i-1], '', operationProbs))
            if i>1 and j>1 and s1[i-1]==s2[j-2] and s1[i-2]==s2[j-1] and s1[i-1]!=s2[j-1]:
                distMatrix[i][j] = min(distMatrix[i][j],
                                distMatrix[i-2][j-2] + operationWeight(s1[i-2:i], s2[j-2:j], operationProbs))
    return distMatrix[-1][-1]

    #### Край на Вашия код. 
    #############################################################################

def generateEdits(q):
    ### помощната функция, generateEdits по зададена заявка генерира всички възможни редакции на разстояние едно от тази заявка.
    ### Вход: заявка като низ q
    ### Изход: Списък от низове на разстояние 1 по Левенщайн-Дамерау от заявката
    ###
    ### В тази функция вероятно ще трябва да използвате азбука, която е дефинирана с alphabet
    ###
    #############################################################################
    #### Начало на Вашия код. На мястото на pass се очакват 10-15 реда
    
    result = []
    for i in range(len(q)+1):
        result.append(q[:i] + q[i+1:]) # deletions
        result.append(q[:i] + q[i:i+2][::-1] + q[i+2:]) # transpositions
        for c in alphabet:
            result.append(q[:i] + c + q[i:]) # insertions
            if i<len(q) and c != q[i]:
                result.append(q[:i] + c + q[i+1:]) # substitutions
    result = set(result)
    result.remove(q)
    return list(result)
        
    #### Край на Вашия код
    #############################################################################


def generateCandidates(query,dictionary,operationProbs):
    ### Започва от заявката query и генерира всички низове НА РАЗСТОЯНИЕ <= 2, за да се получат кандидатите за корекция. Връщат се единствено кандидати, които са в речника dictionary.
        
    ### Вход:
    ###     Входен низ query
    ###     Речник с допустими (правилни) думи: dictionary
    ###     речник с вероятностите на елементарните операции.

    ### Изход:
    ###     Списък от двойки (candidate, candidate_edit_log_probability), където candidate е низ на кандидат, а candidate_edit_log_probability е логаритъм от вероятността за редакция -- минус теглото.
    
    #############################################################################
    #### Начало на Вашия код. На мястото на pass се очакват 10-15 реда

    start = time.time()
    
    allCandidates = [query]
    maxDist = 2
    for i in range(maxDist):
        for x in list(map(generateEdits, allCandidates)):
            allCandidates += x
    allCandidates = set(allCandidates)
    allCandidates.remove(query)
    
    result = list(map(lambda x: (x, editWeight(x, query, operationProbs)), 
                      filter(lambda x: x in dictionary, allCandidates)))

    print(time.time()-start)
    
    return result

    #### Край на Вашия код
    #############################################################################

def correctSpelling(r, dictionary, operationProbs):
    ### Функцията поправя корпус съдържащ евентуално сгрешени думи
    ### Генераторът на кандидати връща и вероятността за редактиране.
    ###
    ### Вход:
    ###    заявка: r - корпус от думи
    ###    речник с правилни думи: dictionary,
    ###    речник с вероятностите на елементарните операции: operationProbs
    ###    Изход: поправен корпус

    #############################################################################
    #### Начало на Вашия код. На мястото на pass се очакват 5-15 реда

    corrected = []
    for i,s in enumerate(r):
        corrected.append([])
        for j,w in enumerate(s):
            if any(map(lambda x: x not in alphabet, w)) or w in dictionary:
                corrected[i].append(w)
                continue
            candidates = generateCandidates(w, dictionary, operationProbs)
            corrected[i].append(min(candidates, key=lambda x: x[1])[0] if candidates else w)
    return corrected

    #### Край на Вашия код
    #############################################################################


In [4]:
import nltk
nltk.download('punkt')
from nltk.corpus import PlaintextCorpusReader
import pickle, time

[nltk_data] Downloading package punkt to /home/psarlov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
start = time.time()
print(editDistance('asdd','asd'))
end = time.time()
print(end-start)

1
0.0003962516784667969


In [8]:
start = time.time()
print(editOperations('заявката','заявьата'))
end = time.time()
print(end-start)

[('з', 'з'), ('а', 'а'), ('я', 'я'), ('в', 'в'), ('к', 'ь'), ('а', 'а'), ('т', 'т'), ('а', 'а')]
0.0003533363342285156


In [9]:
L1 = ['заявката','заявката','заявката','заявката','заявката','заявката']
L2 = ['заявката','заявьата','завякатва','заявкатаа','вя','язвката']
C = [0,1,2,1,7,3]
O = [[('з', 'з'),  ('а', 'а'),  ('я', 'я'),  ('в', 'в'),  ('к', 'к'),  ('а', 'а'),  ('т', 'т'),  ('а', 'а')],
     [('з', 'з'),  ('а', 'а'),  ('я', 'я'),  ('в', 'в'),  ('к', 'ь'),  ('а', 'а'),  ('т', 'т'),  ('а', 'а')],
     [('з', 'з'),  ('а', 'а'),  ('яв', 'вя'),  ('к', 'к'),  ('а', 'а'),  ('т', 'т'),  ('', 'в'),  ('а', 'а')],
     [('з', 'з'),  ('а', 'а'),  ('я', 'я'),  ('в', 'в'),  ('к', 'к'),  ('а', 'а'),  ('т', 'т'),  ('', 'а'),  ('а', 'а')],
     [('з', ''), ('а', ''), ('я', ''), ('в', 'в'), ('к', ''), ('а', ''), ('т', ''), ('а', 'я')],
     [('з', ''), ('а', 'я'), ('я', 'з'), ('в', 'в'),  ('к', 'к'),  ('а', 'а'),  ('т', 'т'),  ('а', 'а')]]
D = [22.75, 32.06, 35.93, 32.02, 62.03, 43.71]

In [10]:
for s1,s2,d in zip(L1,L2,C):
    assert editDistance(s1,s2) == d, "Разстоянието между '{}' и '{}' следва да е '{}'".format(s1,s2,d)
print("Функцията editDistance премина теста.")

Функцията editDistance премина теста.


In [11]:
for s1,s2,o in zip(L1,L2,O):
    assert editOperations(s1,s2) == o, "Операциите редактиращи '{}' до '{}' следва да са '{}'".format(s1,s2,o)
print("Функцията editOperations премина теста.")

Функцията editOperations премина теста.


In [12]:
print('Прочитане на корпуса от текстове...')
corpus_root = '.'
original = PlaintextCorpusReader(corpus_root, 'corpus_original.txt')
fullSentCorpusOriginal = [[w.lower() for w in sent] for sent in original.sents()]
typos = PlaintextCorpusReader(corpus_root, 'corpus_typos.txt')
fullSentCorpusTypos = [[w.lower() for w in sent] for sent in typos.sents()]
print('Готово.')

Прочитане на корпуса от текстове...
Готово.


In [31]:
start = time.time()
print('Пресмятане вероятностите на елементарните операции...')
operationProbs = computeOperationProbs(fullSentCorpusOriginal,fullSentCorpusTypos)
print('Готово.')
ps = [operationProbs[k] for k in operationProbs.keys()]
assert max(ps) < 0.2, "Не би следвало  да има елементарна операция с толкова голяма вероятност."
assert min(ps) > 0, "Използвайте изглаждане."
id_prob = 0
for k in operationProbs.keys():
    if k[0]==k[1]:
        id_prob += operationProbs[k]
assert id_prob > 0.95, "Би следвало операцията идентитет да има най-голяма вероятност."
print("Функцията computeOperationProbs премина теста.")
end = time.time()
print(end-start)

Пресмятане вероятностите на елементарните операции...
Готово.
Функцията computeOperationProbs премина теста.
7.5549094676971436


In [32]:
id_prob

0.9855009376341495

In [40]:
print("Запис на вероятностите във файл probabilities.pkl ...")
opfile = open("probabilities.pkl", "wb")
pickle.dump(operationProbs, opfile)
opfile.close()
print('Готово.')

Запис на вероятностите във файл probabilities.pkl ...
Готово.


In [41]:
for s1,s2,d in zip(L1,L2,D):
    print(d, editWeight(s1,s2,operationProbs))
    assert abs(editWeight(s1,s2,operationProbs) - d) < 1 , "Теглото между '{}' и '{}' следва да е приблизително '{}'".format(s1,s2,d)
print("Функцията editWeight премина теста.")

22.75 22.74648836787877
32.06 32.15199173375289
35.93 35.95728125951019
32.02 32.022027714067235
62.03 62.06029433333477
43.71 43.719280741425045
Функцията editWeight премина теста.


In [42]:
assert len(set(generateEdits("тест"))) == 269, "Броят на елементарните редакции \"тест\"  следва да е 269"
print("Функцията generateEdits премина теста.")

Функцията generateEdits премина теста.


In [43]:
start = time.time()
len(generateEdits("товаемногодългадумаааааааааааааааааааааааааааааааааа"))
end = time.time()
print(end-start)

0.0015959739685058594


In [50]:
dictionary = extractDictionary(fullSentCorpusOriginal)
#### Тест на generate_candidates
assert len(set(generateCandidates("такяива",dictionary,operationProbs))) == 4, "Броят на генерираните кандидати следва да е 4"
print("Функцията generateCandidates премина теста.")

0.14748907089233398
Функцията generateCandidates премина теста.


In [53]:
sorted(generateCandidates("нва",dictionary,operationProbs), key=lambda x: x[1])

0.03992581367492676


[('на', 14.067445863891008),
 ('ва', 14.416622035858103),
 ('нова', 16.409552955577553),
 ('нива', 16.452052269094835),
 ('два', 17.351143578367196),
 ('сва', 17.72583702780861),
 ('ав', 18.99567808319694),
 ('а', 20.509208065254306),
 ('нрав', 21.0096329027081),
 ('нави', 21.031108316433674),
 ('н', 21.37043401544498),
 ('в', 21.719610187412076),
 ('ана', 21.93129265835397),
 ('ра', 22.284492697278857),
 ('та', 22.712586508164904),
 ('ла', 22.782331125518034),
 ('вар', 22.872339056732557),
 ('ван', 23.033359359690593),
 ('явна', 23.24135233809516),
 ('са', 23.25158300889759),
 ('да', 23.443729607763398),
 ('над', 23.499281813477005),
 ('нов', 23.71254110713153),
 ('нас', 23.730919988782638),
 ('за', 23.81842305720481),
 ('ов', 23.84320151370484),
 ('не', 23.852130389602735),
 ('ма', 23.89821017382264),
 ('лв', 23.992733247675808),
 ('вас', 24.080096160749733),
 ('нам', 24.09917150055037),
 ('нрава', 24.294423649832222),
 ('ща', 24.424558860775125),
 ('ча', 24.424558860775125),
 ('га',

In [54]:
#### Тест на correct_spelling
start = time.time()
corr = correctSpelling(fullSentCorpusTypos[3668:3669],dictionary,operationProbs)
assert ' '.join(corr[0]) == 'третата група ( нареченската ) бе ударила на камък : поради курортния характер на селото пръчовете от наречен били още миналата година премахнати , защото замърсявали околната среда със силната си миризма и създавали у чужденците впечатление за първобитност .', "Коригираната заявка следва да е 'третата група ( нареченската ) бе ударила на камък : поради курортния характер на селото пръчовете от наречен били още миналата година премахнати , защото замърсявали околната среда със силната си миризма и създавали у чужденците впечатление за първобитност .'."
print("Функцията correctSpelling премина теста.")
end = time.time()
print(end-start)

0.18945598602294922
0.48256683349609375
0.17010283470153809
0.18068313598632812
0.1049203872680664
0.10046029090881348
0.1784977912902832
0.2162761688232422
0.3988759517669678
Функцията correctSpelling премина теста.
2.107445478439331


In [17]:
' '.join(fullSentCorpusTypos[3668:3669][0])

'третадта група ( наруеченската ) бе ударила на камък : поради курортния хаарктер на селото пръчовете от наречен били още имналата гоаина премахнати , защоот замърсявали околната среда със силнащта си миризма и съзадвали у чужденците впечатление за първобтиност .'

In [18]:
' '.join(corr[0])

'третата група ( нареченската ) бе ударила на камък : поради курортния характер на селото пръчовете от наречен били още миналата година премахнати , защото замърсявали околната среда със силната си миризма и създавали у чужденците впечатление за първобитност .'

In [33]:
start = time.time()
correct = PlaintextCorpusReader(corpus_root, 'corpus_correct.txt')
fullSentCorpusCorrect = [[w.lower() for w in sent] for sent in correct.sents()]
corpus_corrected = correctSpelling(fullSentCorpusCorrect,dictionary,operationProbs)
corpus_corrected = '\n'.join(' '.join(s) for s in corpus_corrected)
with open('corrected.txt', 'w') as f:
    f.write(corpus_corrected)
end = time.time()
print(end-start)

273.874144077301
